<img style="float: left;;" src='Figures/iteso.jpg' width="100" height="200"/></a>

# <center> <font color= #000047> Ancho y Frecuencia Constantes </font> </center>

La discretización de variables continuas sirve para transformar valores numéricos continuos en categorías discretas o rangos, lo que facilita el análisis de datos y el entrenamiento de modelos de aprendizaje automático. Este proceso puede simplificar datos complejos, mejorar la interpretabilidad de modelos y, en algunos casos, optimizar el rendimiento de algoritmos que no manejan bien datos continuos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

N=10**4
normal=np.random.normal(size=(N,1))
gamma=np.random.gamma(shape=4,size=(N,1))
x=0.5*normal+0.5*gamma
y=x**2+1+np.random.uniform(-5,5,(N,1))
df=pd.DataFrame(np.concatenate((x,y),axis=1),columns=('x','y'))
df.hist(bins=50)

In [ ]:
plt.scatter(df['x'],df['y'],s=5)
plt.grid()

In [ ]:
#@title Discretización por ancho constante
bins=50
ancho=(df.x.max()-df.x.min())/bins
#intervalos=np.arange(df.x.min(),df.x.max()+ancho,ancho)
intervalos=np.linspace(df.x.min(),df.x.max(),bins+1)
ancho,intervalos

In [ ]:
pd.cut?

In [ ]:
# Discretización
#
# Se obtienen valores distribuidos a lo largo del rango de 'x' de forma que le asignamos la misma importancia
df['ancho_x']=pd.cut(df.x,intervalos,include_lowest=True)
df.head()

In [ ]:
df['ancho_x'].value_counts()

In [ ]:
df['ancho_x'].astype('str').hist(figsize=(15,5),bins=50,xrot=90)

In [ ]:
x=df.groupby('ancho_x', observed=False)['x'].mean()
y=df.groupby('ancho_x',  observed=False)['y'].mean()
x.isna().sum(),y.isna().sum()

In [ ]:
#x=x.dropna()
#y=y.dropna()
while(x.isna().sum()>0): # Parche
    nans_x=np.where(x.isna())
    for idx in nans_x:
        x.iloc[idx]=x.iloc[idx-1]
        y.iloc[idx]=y.iloc[idx-1]
x.isna().sum()

In [ ]:
y.isna().sum()

In [ ]:
#Regresión
lin_ancho_x=LinearRegression()
lin_ancho_x.fit(x.values.reshape(-1,1),y)
predict_ancho_x=lin_ancho_x.predict(df['x'].values.reshape(-1,1))

In [ ]:
lin_SD=LinearRegression()
lin_SD.fit(df[['x']],df['y'])
predict_SD=lin_SD.predict(df[['x']])

In [ ]:
# Gráfico
plt.scatter(df.x,df.y,s=5)
plt.plot(df.x,predict_SD,'k',label='Regresión sin discretizar')
plt.plot(df.x,predict_ancho_x,'r',label='Regresión con ancho constante')
plt.scatter(x,y,marker='x',s=20,label='Valores discretizados con ancho constante')
plt.legend()
plt.grid()

In [ ]:
# Discretización con frecuencia constante
# Usamos la misma cantidad de contenedores
bins

In [ ]:
df.x.shape[0] # Cantidad de observaciones

In [ ]:
# Cada contenedor tendrá 200 observaciones
df.x.shape[0]/bins

In [ ]:
pd.qcut?

In [ ]:
ancho

In [ ]:
# Discretización
df['freq_x'], intervalos = pd.qcut(df.x,bins, retbins=True)

In [ ]:
df

In [ ]:
df['freq_x'].value_counts()

In [ ]:
xF = df.groupby('freq_x', observed=False)['x'].mean()
yF = df.groupby('freq_x', observed=False)['y'].mean()

In [ ]:
xF.isna().sum(), yF.isna().sum()

In [ ]:
#Regresión
lin_freq_x=LinearRegression()
lin_freq_x.fit(xF.values.reshape(-1,1),yF)
predict_freq_x=lin_freq_x.predict(df[['x']].values)

In [ ]:
# Gráfico
plt.scatter(df.x,df.y,s=5)
plt.plot(df.x,predict_SD,'k',label='Regresión sin discretizar')
plt.plot(df.x,predict_ancho_x,'r',label='Regresión con ancho constante')
plt.plot(df.x,predict_freq_x,'g',label='Regresión con frecuencia constante')
plt.scatter(xF,yF,marker='x',s=20,label='Valores discretizados con frecuencia constante')
plt.legend()
plt.grid()

In [ ]:
# Modelo con datos sin discretizar
lin_SD.coef_, lin_SD.intercept_

In [ ]:
# Modelo con datos discretizados
lin_freq_x.coef_, lin_freq_x.intercept_

# Modelos conseguidos
Modelo con datos sin discretizar
$$
y=?
$$
Modelo con datos discretizados con frecuencia constante
$$
y=?
$$

## Ejemplo 2

In [ ]:
titanic_df = pd.read_csv("titanic.csv")

In [ ]:
titanic_df.info()

In [ ]:
titanic_df['Age'].hist()

In [ ]:
titanic_df['Age'].describe()

In [ ]:
def cut_age(df, cut_values, label_names):
    df["Age_labels"] = pd.cut(df['Age'], bins=cut_values, labels=label_names)
    return df

In [ ]:
cut_values = [0,3,12,19,35,60,80]
labels_names = ['infants', 'children', 'teenagers', 'young adults', 'middle adults', 'seniors']

In [ ]:
cut_age(titanic_df, cut_values, labels_names)

In [ ]:
import seaborn as sns

In [ ]:
sns.catplot(x='Age_labels', row='Survived', kind="count", data=titanic_df, aspect=4, height=3)

In [ ]:
def qcut_fare(df, cut_values, label_names):
    df["Fare_labels"] = pd.qcut(df['Fare'], cut_values, labels=label_names)
    return df
labels=['range1','range2','range3','range4']
titanic_df = qcut_fare(titanic_df, 4, labels)

In [ ]:
titanic_df

In [ ]:
sns.catplot(x='Fare_labels',  kind="count", data=titanic_df, aspect=4, height=3)